In [ ]:
import tqdm
import json
import pandas as pd

SHEET_ID = '1b6h2RApBs2xbN6-eGVvxH68EALKDklvS91fb7d_IVz4'
SHEET_TABS = {
    'REGISTROS': 2103959160,
    'USUARIOS': 102021014,
}
REGISTROS_URL = f'https://docs.google.com/spreadsheets/d/{SHEET_ID}/export?format=csv&gid={SHEET_TABS["REGISTROS"]}'
USUARIOS_URL = f'https://docs.google.com/spreadsheets/d/{SHEET_ID}/export?format=csv&gid={SHEET_TABS["USUARIOS"]}'

# Vamos a hacer un grafo por carrera... hay que setupearla aca!
# sistemas informatica agrimensura alimentos civil electricista electronica 
# industrial mecanica naval petroleo quimica sistemasviejo
CARRERA = 'informatica'

In [ ]:
# Empezamos cargando todos los datos (los usuarios y los mapas estan en dos tablas distintas)
df_registros = pd.read_csv(REGISTROS_URL)
df_registros.drop("Timestamp", axis=1, inplace=True)
df_registros.dropna(inplace=True, how="any")
df_registros = df_registros[df_registros['Carrera'] == CARRERA]
display(df_registros.sample(3))

df_usuarios = pd.read_csv(USUARIOS_URL)
df_usuarios.drop("Timestamp", axis=1, inplace=True)
df_usuarios.dropna(inplace=True, how="all")
df_usuarios = df_usuarios[df_usuarios['Carrera'] == CARRERA]
display(df_usuarios.sample(3))

In [ ]:
### Testing purposes: en vez de laburar con toooodos los datos, laburemos solo con los integrantes del grupo
# df_usuarios = df_usuarios[df_usuarios['Padron'].isin(["100029", "101696", "101109", "0000"])]

In [ ]:
# Mergeamos!
df_mergeado = df_usuarios.merge(df_registros)
df_mergeado.sample(3)

In [ ]:
df_mergeado.describe()

In [ ]:
# Ahora, desempaquetamos la columna "Mapa" (un JSON que contiene notas, metadata, de todo...) 
#   y finalmente lleguemos al df que vamos a usar durante todo el tp

df = df_mergeado
new_df = pd.DataFrame()
for idx in tqdm.tqdm(df.index):
    json_mapa = json.loads(df.loc[idx]['Mapa'])
    
    for k in json_mapa.keys():
        if not json_mapa[k]: 
            continue
        if k == "materias":
            df_json = pd.json_normalize(json_mapa, k, record_prefix="materia_")
        else:
            df_json = pd.DataFrame({k: [json_mapa[k]]})
        df_json["Padron"] = df.loc[idx]['Padron']
        new_df = pd.concat([new_df, df_json])

df = df.merge(new_df, how="outer")
df.drop('Mapa', axis=1, inplace=True)
# Hace mucho tiempo, el fiuba map tenia un feature que se llamaba "materia_cuatri" (en vez del nuevo "materia_cuatrimestre")
# En donde el usuario seteaba en cuantos N cuatris iba a cursar X materia, en vez de setear el cuatri exacto
# Como ya no se usa en el upstream, no es lo correcto que lo utilicemos en el analisis de datos
if 'materia_cuatri' in df.columns: df.drop('materia_cuatri', axis=1, inplace=True)
df.sample(3)

In [ ]:
# Exportamos todo a un hermoso pickle
df.to_pickle('fiuba-map-data.pickle')